# Level Up to Agentic AI: Roast My Resume Agent

### IEEE IGDTUW Workshop — SOLUTIONS

**This notebook has all TODO cells filled in.** Use it as a reference if you get stuck, or to catch up if you fall behind.

**3 Levels:**
- **Level 1** — Teach Your AI to Think (Structured Output)
- **Level 2** — Give Your AI Superpowers (Tool Use)
- **Level 3** — Make Your AI Argue With Itself (Self-Correction)

**+ Bonus:** Deploy it as a web app, ATS scoring, LinkedIn post generator

---

## Setup

**Before running this cell:**
1. Go to <a href="https://console.groq.com/keys" target="_blank" rel="noopener noreferrer">console.groq.com/keys</a>
2. Create a new Groq API key
3. Copy the key
4. In Colab, click the **key icon** (Secrets) in the left sidebar
5. Add a new secret: Name = `GROQ_API_KEY`, Value = your key
6. Toggle the **notebook access** switch ON

**Running locally?** Copy `.env.example` to `.env` and set your key there instead.

Then run the cell below:

In [ ]:
# ============================================================
# SETUP — Run this cell first!
# ============================================================
!pip install -q -U groq python-dotenv

import os
import json
import time
from dotenv import load_dotenv
from groq import Groq

try:
    from google.colab import userdata
except Exception:
    userdata = None

load_dotenv()

def _get_groq_key():
    key = os.getenv('GROQ_API_KEY')
    if not key and userdata is not None:
        try:
            key = userdata.get('GROQ_API_KEY')
        except Exception:
            pass
    if not key:
        raise ValueError('GROQ_API_KEY not found. Add it to Colab Secrets or .env.')
    return key

class _GroqResponse:
    def __init__(self, text):
        self.text = text

# ── Available Groq models (pick one) ──────────────────────────
GROQ_MODELS = {
    'llama-3.3-70b-versatile':  '70B params — best for structured JSON, strong instruction-following',
    'qwen-3-32b':               '32B params — great reasoning, multilingual',
    'openai/gpt-oss-20b':       '20B params — lightweight & fast',
    'openai/gpt-oss-120b':      '120B params — most capable, may have lower free-tier limits',
    'meta-llama/llama-4-scout-17b-16e-instruct': '17B MoE — newest Llama, tool-use capable',
}

class GroqModelAdapter:
    def __init__(self, model_name='llama-3.3-70b-versatile'):
        self.client = Groq(api_key=_get_groq_key())
        self.model_name = model_name

    def generate_content(self, prompt):
        completion = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{'role': 'user', 'content': prompt}],
            temperature=0.3,
        )
        text = (completion.choices[0].message.content or '').strip()
        return _GroqResponse(text)

# Configure the API
# ── Choose your model (change here to try a different one) ────
SELECTED_MODEL = os.getenv('GROQ_MODEL', 'llama-3.3-70b-versatile')
model = GroqModelAdapter(SELECTED_MODEL)
print(f"Using model: {SELECTED_MODEL}")
print(f"Available models: {', '.join(GROQ_MODELS.keys())}\n")

# Quick test — if this prints a response, you're good!
test = model.generate_content("Say 'Setup complete!' and nothing else.")
print(f"[OK] {test.text.strip()}")
print("\nYou're ready to build your agent.")

### Check rate limit / quota (optional)

Groq lets you monitor usage from the console. You can:
- **Dashboard:** Open <a href="https://console.groq.com" target="_blank" rel="noopener noreferrer">console.groq.com</a> (same account as your API key) to see RPM/TPM/RPD usage.
- **Probe:** Run the next cell to make one test request — if it succeeds, you have quota; if 429, it will show "retry in Xs".

In [ ]:
# ============================================================
# CHECK QUOTA / RATE LIMIT (run in Colab)
# ============================================================
# Official dashboard (open in browser — same Groq account as API key):
print("Rate limit dashboard: https://console.groq.com")
print("Or: https://console.groq.com\n")

# Probe: one request to see if you have quota right now
import re
try:
    r = model.generate_content("Reply with one word: OK")
    print("[OK] You have quota. Response:", r.text.strip()[:50])
except Exception as e:
    err = str(e)
    print("[429 / quota] Request failed.")
    if "retry in" in err.lower():
        sec = re.search(r"retry in (\d+\.?\d*)s", err, re.I)
        if sec:
            print(f"  → Wait {float(sec.group(1)):.0f}s and try again.")
    if "limit: 0" in err:
        print("  → Check your Groq usage in the console and verify GROQ_API_KEY in Secrets/.env.")

---

## Sample Resume Bullets

Pick one to roast (or use your own!):

In [ ]:
# ============================================================
# SAMPLE RESUME BULLETS — Pick one or write your own!
# ============================================================

samples = [
    "Proficient in MS Office and team management",
    "Worked on various projects using Python",
    "Good communication skills",
    "Responsible for handling database operations",
    "Participated in college hackathon and won prize",
    "Familiar with machine learning concepts",
    "Did internship at a startup and learned many things",
    "Team player with leadership qualities",
]

# === PICK YOUR BULLET ===
resume_bullet = samples[0]

print(f"[NOTE] Your bullet: \"{resume_bullet}\"")
print("\nLet's roast it.")

---

## LEVEL 1: Teach Your AI to Think

Most people use LLMs like this: *"Write me something"* → get a wall of text.

Agents think differently: *"Return structured data"* → get JSON you can actually use in code.

### SOLUTION

In [ ]:
# ============================================================
# LEVEL 1: Structured Roast — SOLUTION
# ============================================================

prompt = f"""You are a brutally honest resume reviewer. You're funny but constructive.
Analyze this resume bullet point and return ONLY valid JSON — no markdown, no explanation.

Resume bullet: "{resume_bullet}"

Return JSON with these exact keys:
- "score": integer from 1 to 10 (10 = perfect, 1 = delete immediately)
- "roast": a funny but honest critique in 2-3 sentences (be savage but helpful)
- "rewrite": an improved version of the bullet point (start with an action verb, include metrics if possible)
- "keywords_missing": a list of 3-5 strong keywords that should be added to make this bullet stand out
"""

# --- Don't modify below this line ---
response = model.generate_content(prompt)
text = response.text.strip()
if text.startswith('```'):
    text = text.split('\n', 1)[1]
if text.endswith('```'):
    text = text.rsplit('```', 1)[0]

result_1 = json.loads(text.strip())

# Pretty display
print(f"[TARGET] Score: {result_1['score']}/10")
print(f"\n[FIRE] Roast: {result_1['roast']}")
print(f"\n[SPARKLE] Rewrite: {result_1['rewrite']}")
print(f"\n[KEY] Missing Keywords: {', '.join(result_1['keywords_missing'])}")

**Agentic Pattern #1: Structured Output** [CHECK]

---

## LEVEL 2: Give Your AI Superpowers

### The Tool

In [ ]:
# ============================================================
# THE TOOL — Pre-built (just run this cell)
# ============================================================

def get_role_requirements(role: str) -> str:
    """
    Returns what recruiters look for in a specific role.
    This is your agent's 'superpower' — external knowledge
    that the LLM doesn't have on its own.
    """
    requirements = {
        "software engineer": (
            "Strong DSA fundamentals, system design, clean code practices, "
            "CI/CD experience, testing frameworks, specific language/framework "
            "expertise (not just 'Python'), quantified impact metrics (%, $, users), "
            "open source contributions"
        ),
        "data scientist": (
            "Statistical modeling, Python/R proficiency, SQL mastery, "
            "ML frameworks (scikit-learn, PyTorch), A/B testing design, "
            "business impact quantification, data pipeline experience, "
            "visualization skills"
        ),
        "product manager": (
            "User research methodology, metrics-driven decisions (DAU, retention, conversion), "
            "roadmap planning, stakeholder management, market analysis, A/B testing, PRD writing"
        ),
        "ml engineer": (
            "Model training and fine-tuning, MLOps (MLflow, Kubeflow), "
            "distributed training, model serving (TensorRT, ONNX), "
            "data pipeline engineering, experiment tracking, production deployment"
        ),
        "frontend developer": (
            "React/Vue/Angular expertise, performance optimization (Core Web Vitals), "
            "accessibility (WCAG), responsive design, state management, "
            "testing (Jest, Cypress), design system experience"
        ),
        "backend developer": (
            "API design (REST/GraphQL), database optimization (SQL + NoSQL), "
            "caching strategies, message queues, microservices architecture, "
            "monitoring/observability, load testing"
        ),
        "devops engineer": (
            "CI/CD pipeline design, IaC (Terraform, Pulumi), container orchestration (K8s), "
            "monitoring (Prometheus, Grafana), cloud platforms (AWS/GCP/Azure), "
            "security best practices, incident response"
        ),
    }

    role_lower = role.lower()
    for key, value in requirements.items():
        if key in role_lower:
            return value
    return (
        "Quantified achievements, specific technologies, measurable impact, "
        "action verbs, industry-specific keywords, problem-solving examples"
    )

# --- Test the tool ---
target_role = "Software Engineer"
context = get_role_requirements(target_role)

print(f"[TOOL] Tool called: get_role_requirements('{target_role}')")
print(f"\n[BRIEFCASE] Recruiters look for: {context}")

### SOLUTION

In [ ]:
# ============================================================
# LEVEL 2: Tool-Powered Roast — SOLUTION
# ============================================================

prompt_2 = f"""You are a resume expert who knows exactly what recruiters want.

For a {target_role} role, recruiters specifically look for:
{context}

Analyze this resume bullet point with that context in mind:
"{resume_bullet}"

Return ONLY valid JSON — no markdown, no explanation — with these exact keys:
- "score": integer 1-10 (how well does this bullet match what recruiters want?)
- "roast": brutal but constructive critique that references specific recruiter expectations (2-3 sentences)
- "rewrite": rewritten bullet optimized for {target_role} applications (start with action verb, include metrics)
- "missing_for_role": a string describing what key skills/keywords are missing based on the recruiter expectations
"""

# --- Don't modify below this line ---
response_2 = model.generate_content(prompt_2)
text_2 = response_2.text.strip()
if text_2.startswith('```'):
    text_2 = text_2.split('\n', 1)[1]
if text_2.endswith('```'):
    text_2 = text_2.rsplit('```', 1)[0]

result_2 = json.loads(text_2.strip())

# Pretty display
print(f"[TARGET] Score: {result_2['score']}/10")
print(f"\n[FIRE] Roast: {result_2['roast']}")
print(f"\n[SPARKLE] Rewrite: {result_2['rewrite']}")
print(f"\n[BRIEFCASE] Missing for {target_role}: {result_2['missing_for_role']}")

**Agentic Pattern #2: Tool Use** [CHECK]

---

## LEVEL 3: Make Your AI Argue With Itself

### SOLUTION

In [ ]:
# ============================================================
# LEVEL 3: The Self-Critique Loop — SOLUTION
# ============================================================

rewrite_from_level_2 = result_2['rewrite']

prompt_3 = f"""You are a senior hiring manager at a FAANG company (Google, Meta, Amazon, etc.).
You have reviewed thousands of resumes and know exactly what stands out.

A junior resume writer produced this bullet point for a {target_role} resume:
"{rewrite_from_level_2}"

For context, recruiters for {target_role} roles look for:
{context}

Your job:
1. Find EXACTLY 3 specific problems with this rewrite (be precise, not vague)
2. Fix all 3 problems and produce the FINAL, perfected version
3. Rate your confidence that this final version would pass ATS and impress a recruiter

Return ONLY valid JSON — no markdown, no explanation — with:
- "problems": a list of exactly 3 strings, each describing a specific issue
- "final_version": the ultimate perfected bullet point
- "confidence_score": integer 1-10 (10 = guaranteed interview callback)
"""

# --- Don't modify below this line ---
response_3 = model.generate_content(prompt_3)
text_3 = response_3.text.strip()
if text_3.startswith('```'):
    text_3 = text_3.split('\n', 1)[1]
if text_3.endswith('```'):
    text_3 = text_3.rsplit('```', 1)[0]

result_3 = json.loads(text_3.strip())

# Pretty display
print("[THINKING] Self-Critique Results:")
print("\nProblems found:")
for i, problem in enumerate(result_3['problems'], 1):
    print(f"  {i}. {problem}")

print(f"\n[OK] Final Version: {result_3['final_version']}")
print(f"\n[STATS] Confidence: {result_3['confidence_score']}/10")

# === THE BEFORE / AFTER ===
print("\n" + "="*50)
print("[NOTE] BEFORE vs AFTER")
print("="*50)
print(f"\n[X] Before: \"{resume_bullet}\"")
print(f"\n[OK] After:  \"{result_3['final_version']}\"")
print("\n" + "="*50)

**Agentic Pattern #3: Self-Correction** [CHECK]

---

## The Full Pipeline

In [ ]:
# ============================================================
# HELPER FUNCTIONS
# ============================================================

def stage_print(stage, icon, color, message):
    """Print a visually distinct stage header."""
    colors = {
        'cyan': '\033[96m',
        'red': '\033[91m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'magenta': '\033[95m',
        'white': '\033[97m',
        'reset': '\033[0m'
    }
    c = colors.get(color, colors['white'])
    r = colors['reset']
    print(f"\n{c}{'='*60}{r}")
    try:
        # Try printing with icon
        print(f"{c}{icon}  [ {stage} ]{r}")
    except UnicodeEncodeError:
        # Fallback if terminal doesn't support emoji
        print(f"{c}[ {stage} ]{r}")
    print(f"{c}{'='*60}{r}")
    time.sleep(0.5)
    print(f"\n{message}\n")


def thinking_dots(text, duration=1.5):
    """Simulate agent thinking."""
    import sys
    sys.stdout.write(f"\033[90m{text}")
    for _ in range(3):
        time.sleep(duration / 3)
        sys.stdout.write(".")
        sys.stdout.flush()
    print("\033[0m")


def get_role_requirements(role: str) -> str:
    """TOOL: Returns what recruiters look for in a specific role."""
    requirements = {
        "software engineer": "Strong DSA fundamentals, system design, clean code practices, CI/CD experience, testing frameworks, specific language/framework expertise (not just 'Python'), quantified impact metrics (%, $, users), open source contributions",
        "data scientist": "Statistical modeling, Python/R proficiency, SQL mastery, ML frameworks (scikit-learn, PyTorch), A/B testing design, business impact quantification, data pipeline experience, visualization skills",
        "product manager": "User research methodology, metrics-driven decisions (DAU, retention, conversion), roadmap planning, stakeholder management, market analysis, A/B testing, PRD writing",
        "ml engineer": "Model training and fine-tuning, MLOps (MLflow, Kubeflow), distributed training, model serving (TensorRT, ONNX), data pipeline engineering, experiment tracking, production deployment",
        "frontend developer": "React/Vue/Angular expertise, performance optimization (Core Web Vitals), accessibility (WCAG), responsive design, state management, testing (Jest, Cypress), design system experience",
        "backend developer": "API design (REST/GraphQL), database optimization (SQL + NoSQL), caching strategies, message queues, microservices architecture, monitoring/observability, load testing",
        "devops engineer": "CI/CD pipeline design, IaC (Terraform, Pulumi), container orchestration (K8s), monitoring (Prometheus, Grafana), cloud platforms (AWS/GCP/Azure), security best practices, incident response"
    }
    role_lower = role.lower()
    for key, value in requirements.items():
        if key in role_lower:
            return value
    return "Quantified achievements, specific technologies, measurable impact, action verbs, industry-specific keywords, problem-solving examples"


def call_llm(prompt, retries=4, delay=10):
    """Call Groq and return parsed JSON with retry logic and rate-limit backoff."""
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            text = response.text.strip()
            if text.startswith('```'):
                text = text.split('\n', 1)[1]
            if text.endswith('```'):
                text = text.rsplit('```', 1)[0]
            return json.loads(text.strip())
        except Exception as e:
            err_str = str(e).lower()
            is_rate_limit = "429" in str(e) or "resource exhausted" in err_str or "quota" in err_str
            if is_rate_limit and attempt < retries - 1:
                print(f"\n\033[93mRate limit hit. Retrying in {delay}s... (attempt {attempt+1}/{retries})\033[0m")
                time.sleep(delay)
                delay = min(delay * 2, 65)  # Exponential backoff, cap at 65s
            else:
                if is_rate_limit:
                    print("\033[91mQuota exceeded. Try a different model from GROQ_MODELS in Setup, or verify GROQ_API_KEY.\033[0m")
                raise e


def roast_my_resume(bullet: str, role: str) -> dict:
    """Run the full Level 1 → 2 → 3 pipeline for a single bullet and role. Used by Gradio."""
    context = get_role_requirements(role)
    # Level 1: structured roast
    p1 = f"""You are a brutally honest resume reviewer. You're funny but constructive.
Analyze this resume bullet point and return ONLY valid JSON — no markdown, no explanation.

Resume bullet: "{bullet}"

Return JSON with these exact keys:
- \"score\": integer from 1 to 10 (10 = perfect, 1 = delete immediately)
- \"roast\": a funny but honest critique in 2-3 sentences (be savage but helpful)
- \"rewrite\": an improved version of the bullet point (start with an action verb, include metrics if possible)
- \"keywords_missing\": a list of 3-5 strong keywords that should be added to make this bullet stand out
"""
    analysis = call_llm(p1)
    # Level 2: tool-powered rewrite
    p2 = f"""You are a resume expert who knows exactly what recruiters want.

For a {role} role, recruiters specifically look for:
{context}

Analyze this resume bullet point with that context in mind:
"{bullet}"

Return ONLY valid JSON — no markdown, no explanation — with these exact keys:
- \"score\": integer 1-10 (how well does this bullet match what recruiters want?)
- \"roast\": brutal but constructive critique that references specific recruiter expectations (2-3 sentences)
- \"rewrite\": rewritten bullet optimized for {role} applications (start with action verb, include metrics)
- \"missing_for_role\": a string describing what key skills/keywords are missing based on the recruiter expectations
"""
    result_2 = call_llm(p2)
    # Level 3: self-critique
    rewrite = result_2['rewrite']
    p3 = f"""You are a senior hiring manager at a FAANG company (Google, Meta, Amazon, etc.).
You have reviewed thousands of resumes and know exactly what stands out.

A junior resume writer produced this bullet point for a {role} resume:
"{rewrite}"

For context, recruiters for {role} roles look for:
{context}

Your job:
1. Find EXACTLY 3 specific problems with this rewrite (be precise, not vague)
2. Fix all 3 problems and produce the FINAL, perfected version
3. Rate your confidence that this final version would pass ATS and impress a recruiter

Return ONLY valid JSON — no markdown, no explanation — with:
- \"problems\": a list of exactly 3 strings, each describing a specific issue
- \"final_version\": the ultimate perfected bullet point
- \"confidence_score\": integer 1-10 (10 = guaranteed interview callback)
"""
    result_3 = call_llm(p3)
    return {
        'original': bullet,
        'initial_score': analysis['score'],
        'roast': analysis['roast'],
        'problems_found': result_3['problems'],
        'final_version': result_3['final_version'],
        'confidence': result_3['confidence_score'],
    }


print("[OK] Helpers loaded.")

---

# BONUS LEVELS

---

## BONUS: ATS Score Gamification

In [ ]:
# ============================================================
# ATS SCORE — Before vs After
# ============================================================

original_bullet = resume_bullet
final_bullet = result_3['final_version']

ats_prompt = f"""You are an Applicant Tracking System (ATS) scanner.
Score these two resume bullets for ATS compatibility for a {target_role} role.

Key factors: action verbs, quantified metrics, relevant keywords,
specificity, industry standard terminology.

Recruiters for {target_role} look for: {context}

Bullet A (original): "{original_bullet}"
Bullet B (improved): "{final_bullet}"

Return ONLY valid JSON with:
- "original_ats_score": integer 1-100
- "final_ats_score": integer 1-100
- "improvement": percentage improvement as a string (e.g., "+65%")
- "ats_tips": list of 3 tips for even higher ATS compatibility
"""

response_ats = model.generate_content(ats_prompt)
text_ats = response_ats.text.strip()
if text_ats.startswith('```'):
    text_ats = text_ats.split('\n', 1)[1]
if text_ats.endswith('```'):
    text_ats = text_ats.rsplit('```', 1)[0]
ats_result = json.loads(text_ats.strip())

# Display
print("[ROBOT] ATS COMPATIBILITY SCAN")
print("=" * 40)
print(f"\n[X] Original: {ats_result['original_ats_score']}/100")
bar_orig = "\u2588" * (ats_result['original_ats_score'] // 5) + "\u2591" * (20 - ats_result['original_ats_score'] // 5)
print(f"   [{bar_orig}]")

print(f"\n[OK] Final:    {ats_result['final_ats_score']}/100")
bar_final = "\u2588" * (ats_result['final_ats_score'] // 5) + "\u2591" * (20 - ats_result['final_ats_score'] // 5)
print(f"   [{bar_final}]")

print(f"\n[CHART] Improvement: {ats_result['improvement']}")
print(f"\n[TIP] Tips for even higher scores:")
for tip in ats_result['ats_tips']:
    print(f"   \u2022 {tip}")

---

## SECRET LEVEL 4: Deploy as a Web App

In [ ]:
# ============================================================
# SECRET LEVEL 4: Deploy with Gradio
# ============================================================
!pip install -q gradio

import gradio as gr

def roast_app(bullet: str, role: str) -> str:
    """Wrapper for the Gradio interface."""
    try:
        result = roast_my_resume(bullet, role)
        output = f"""## Results\n
**Original:** \"{result['original']}\"\n
**Initial Score:** {result['initial_score']}/10\n
**Roast:** {result['roast']}\n
---\n
**Problems Found:**\n"""
        for i, p in enumerate(result['problems_found'], 1):
            output += f"{i}. {p}\n"
        output += f"""\n---\n
**Final Version:** \"{result['final_version']}\"\n
**Confidence:** {result['confidence']}/10
"""
        return output
    except Exception as e:
        return f"Error: {str(e)}. Try again!"


demo = gr.Interface(
    fn=roast_app,
    inputs=[
        gr.Textbox(label="Resume Bullet", placeholder="Paste your resume bullet here..."),
        gr.Dropdown(
            label="Target Role",
            choices=["Software Engineer", "Data Scientist", "Product Manager",
                     "ML Engineer", "Frontend Developer", "Backend Developer",
                     "DevOps Engineer"],
            value="Software Engineer"
        ),
    ],
    outputs=gr.Markdown(label="Agent Output"),
    title="[FIRE] Roast My Resume Agent",
    description="An AI agent that roasts your resume, rewrites it, and argues with itself to make it perfect.",
    examples=[
        ["Proficient in MS Office and team management", "Software Engineer"],
        ["Familiar with machine learning concepts", "ML Engineer"],
        ["Good communication skills", "Product Manager"],
    ],
)

demo.launch(share=True)

---

## BONUS: LinkedIn Post Generator

In [ ]:
# ============================================================
# LINKEDIN POST GENERATOR
# ============================================================

linkedin_prompt = """You are a LinkedIn post writer for a college student who just
built their first AI agent at a workshop.

Write a professional but enthusiastic LinkedIn post (150-200 words) about:
- They attended the "Level Up to Agentic AI" workshop by IEEE IGDTUW
- Mentored by Ishan (AI Engineer, GDG Cloud New Delhi speaker)
- They built a "Roast My Resume" AI Agent from scratch in Python
- The agent uses 3 agentic patterns: Structured Output, Tool Calling, Self-Correction
- They deployed it as a live web app using Gradio
- They're excited about AI Engineering and Agentic AI

Make it sound genuine, not salesy. Include 3-4 relevant hashtags.
Don't use excessive emojis — keep it professional.

Return ONLY valid JSON with:
- "post": the full LinkedIn post text
- "hashtags": list of hashtags used
"""

response_li = model.generate_content(linkedin_prompt)
text_li = response_li.text.strip()
if text_li.startswith('```'):
    text_li = text_li.split('\n', 1)[1]
if text_li.endswith('```'):
    text_li = text_li.rsplit('```', 1)[0]
li_result = json.loads(text_li.strip())

print("[NOTE] Your LinkedIn Post (copy-paste ready):\n")
print("=" * 50)
print(li_result['post'])
print("=" * 50)
print("\n[CLIP] Just copy the text above and paste it into LinkedIn!")

---

## What You Built Today

| Pattern | What It Does | Where It's Used in Production |
|---------|-------------|------------------------------|
| **Structured Output** | LLM returns JSON, not prose | Every AI agent ever |
| **Tool Use** | Agent calls functions for external data | Google Search, API calls, DB queries |
| **Self-Correction** | Agent critiques and fixes its own output | Devin, AutoGPT, production pipelines |

### Where to Go Next

- **LangGraph** — Build multi-agent systems with complex orchestration
- **MCP (Model Context Protocol)** — Universal standard for agent tools
- **Vertex AI / Cloud Run** — Deploy agents to production 24/7
- **Groq Console** — Your API key is free. Keep building.

### You just built what companies pay ₹20L+ for. Keep building.

---

*Workshop by Ishan — AI Engineer | GDG Cloud New Delhi*